In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

E0000 00:00:1758744520.402359 2662279 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


**read about reducer functions**

In [3]:
from typing import TypedDict
from typing_extensions import Annotated
from langgraph.graph import StateGraph, START, END
#reducer functio -> adds some kind of validation to messages
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {'messages': [llm.invoke(state['messages'])]}

builder = StateGraph(State)
builder.add_node('chatbot_node', chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [4]:
message = {'role': 'user', 'content':'Who is the current president of the United States?'}
response = graph.invoke({'messages':[message]})

response["messages"]

[HumanMessage(content='Who is the current president of the United States?', additional_kwargs={}, response_metadata={}, id='7228b767-95ce-42f2-8763-a30d488beab9'),
 AIMessage(content='The current president of the United States is **Joe Biden**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--44592b85-d3a5-416f-bcd5-834c7177bd22-0', usage_metadata={'input_tokens': 11, 'output_tokens': 64, 'total_tokens': 75, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 52}})]

In [5]:
state = None

while True:
    in_message = input("You : ")

    if in_message.lower() in {"quit", "exit"}:
        break

    if state == None:
        state: State = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})


    state = graph.invoke(state)
    print("Bot: ", state["messages"][-1].content)

Bot:  Hello! How can I help you today?
Bot:  To find the square of 69, you multiply 69 by itself:

69 * 69 = 4761

So, the square of 69 is **4761**.
Bot:  As an AI, I don't have personal feelings or opinions like "liking" or "disliking" things.

However, I can tell you that Siri is a very well-known and widely used AI assistant, especially popular on Apple devices. It's designed to help users with a variety of tasks through voice commands, which is a great use of AI technology to make devices more accessible and functional.

We are both examples of AI designed to assist users, just in different ways and often on different platforms!
Bot:  I am a large language model, trained by Google.

My purpose is to provide information, answer your questions, help with creative tasks, and assist you in various ways through text-based communication. I don't have a name, personal feelings, or a physical form.
